In [1]:
import pandas as pd
from imblearn.over_sampling import SMOTE

In [2]:
df = pd.read_excel(r'C:\Users\Lenovo\Desktop\ML_health\Depression_ML_ITEMS.xlsx')
df.head()

,Number,Residence,Hukou,Estates,Debt,Income,Gender,Age,Education,Marriage,...,WAFCS8,WAFCS9,WAFCS10,Chronic1,Chronic2,Chronic3,Chronic4,Chronic5,Chronic6,Depression
0,1,2,2,2,1,7,2,1,6,0,...,-1.03774,-1.04425,-1.04324,-0.22618,0.53747,-0.55702,-0.81609,-0.65780,0.47164,0
1,2,2,2,1,0,0,2,1,6,0,...,-1.03774,-1.04425,-1.04324,-0.97408,-0.23181,-0.55702,-0.04902,-0.65780,-0.99348,0
2,3,2,2,3,1,1,2,4,3,1,...,-0.33977,-0.29348,-0.29708,0.52172,-1.00109,-0.55702,-0.81609,-0.65780,-0.26092,0
3,4,1,1,1,0,2,2,0,2,0,...,-1.03774,-1.04425,-1.04324,0.52172,-0.23181,1.29179,0.71804,1.01182,0.47164,1
4,5,2,2,0,0,0,1,1,6,0,...,-1.03774,-1.04425,-1.04324,1.26962,-0.23181,2.21620,0.71804,1.84663,1.20420,1


In [3]:
X = df.drop('Depression', axis = 1)
X.shape

(11031, 112)

In [4]:
y=df['Depression']
y.shape

(11031,)

In [5]:
pd.Series(y).value_counts()

Depression
0    8832
1    2199
Name: count, dtype: int64

In [6]:
smote = SMOTE()

In [7]:
X.dtypes

Number         int64
Residence      int64
Hukou          int64
Estates        int64
Debt           int64
              ...   
Chronic2     float64
Chronic3     float64
Chronic4     float64
Chronic5     float64
Chronic6     float64
Length: 112, dtype: object

In [8]:
X = X.apply(pd.to_numeric, errors='coerce')

In [9]:
X.dtypes

Number         int64
Residence      int64
Hukou          int64
Estates        int64
Debt           int64
              ...   
Chronic2     float64
Chronic3     float64
Chronic4     float64
Chronic5     float64
Chronic6     float64
Length: 112, dtype: object

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [11]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [12]:
print(y_train.value_counts())  # 原始训练集的类别分布
print(y_train_resampled.value_counts())  # 过采样后的类别分布

Depression
0    7093
1    1731
Name: count, dtype: int64
Depression
1    7093
0    7093
Name: count, dtype: int64


In [13]:
df2=pd.concat([y_train_resampled, X_train_resampled], axis=1)

In [14]:
df2.to_excel(r'C:\Users\Lenovo\Desktop\ML_health\Depression_ML_ITEMS_smote.xlsx',index = False)

In [15]:
df3 = pd.read_excel(r'C:\Users\Lenovo\Desktop\ML_health\Depression_ML_ITEMS_smote.xlsx')

In [16]:
X = df3.drop('Depression', axis = 1)
X.shape

(14186, 112)

In [17]:
y=df3['Depression']
y.shape

(14186,)

In [18]:
X = df3.drop(['Depression', 'Number'], axis=1)

In [19]:
X.shape

(14186, 111)

In [20]:
y=df3['Depression']
y.shape

(14186,)

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [22]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

In [23]:
# 使用训练好的模型对验证集进行预测
y_pred = model.predict(X_val)

# 验证集的真实标签
y_true = y_val

In [24]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.81      0.78      0.80      1457
           1       0.78      0.81      0.80      1381

    accuracy                           0.80      2838
   macro avg       0.80      0.80      0.80      2838
weighted avg       0.80      0.80      0.80      2838

[[1141  316]
 [ 261 1120]]
Accuracy: 0.7966878083157153
Precision: 0.7968908324359549
Recall: 0.7970612543902765
F1 Score: 0.7966766756028177
ROC AUC: 0.8663673633292698
Specificity: 0.7831159917638985


In [25]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [26]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [27]:
# 使用训练好的模型对验证集进行预测
y_pred = model.predict(X_val)

# 验证集的真实标签
y_true = y_val

In [28]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.88      0.85      0.87      1457
           1       0.85      0.88      0.86      1381

    accuracy                           0.87      2838
   macro avg       0.87      0.87      0.87      2838
weighted avg       0.87      0.87      0.87      2838

[[1243  214]
 [ 167 1214]]
Accuracy: 0.8657505285412262
Precision: 0.8658501698551762
Recall: 0.8660979952955021
F1 Score: 0.8657365091502313
ROC AUC: 0.8660979952955022
Specificity: 0.8531228551818806


In [29]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [30]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(X_train, y_train)

KNeighborsClassifier()

In [31]:
# 使用训练好的模型对验证集进行预测
y_pred = model.predict(X_val)

# 验证集的真实标签
y_true = y_val

In [32]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.94      0.80      0.86      1457
           1       0.81      0.95      0.88      1381

    accuracy                           0.87      2838
   macro avg       0.88      0.87      0.87      2838
weighted avg       0.88      0.87      0.87      2838

[[1160  297]
 [  73 1308]]
Accuracy: 0.8696264975334743
Precision: 0.8778740402179928
Recall: 0.871648119865793
F1 Score: 0.8692709720953047
ROC AUC: 0.9474372017134192
Specificity: 0.7961564859299931


In [33]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [34]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [35]:
# 使用训练好的模型对验证集进行预测
y_pred = model.predict(X_val)

# 验证集的真实标签
y_true = y_val

In [36]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.86      0.85      0.85      1457
           1       0.84      0.86      0.85      1381

    accuracy                           0.85      2838
   macro avg       0.85      0.85      0.85      2838
weighted avg       0.85      0.85      0.85      2838

[[1237  220]
 [ 200 1181]]
Accuracy: 0.8520084566596194
Precision: 0.851895231410907
Recall: 0.8520911060340924
F1 Score: 0.8519508122360301
ROC AUC: 0.924601799994732
Specificity: 0.8490048043925875


In [37]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [38]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

In [39]:
param_grid = {
    'n_estimators': [300],
    'learning_rate': [0.05],
    'num_leaves': [61]
}

# 创建 LightGBM 分类器
gbm = LGBMClassifier()

# 创建网格搜索对象
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

# 执行网格搜索
grid_search.fit(X_train, y_train)

# 最佳参数和最佳模型
print("Best parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[LightGBM] [Info] Number of positive: 4570, number of negative: 4508
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012826 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 24315
[LightGBM] [Info] Number of data points in the train set: 9078, number of used features: 111
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503415 -> initscore=0.013660
[LightGBM] [Info] Start training from score 0.013660
[LightGBM] [Info] Number of positive: 4569, number of negative: 4509
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011198 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 24316
[LightGBM] [Info] Number of data points in the train set: 9078, number of used features: 111
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503305 -> initscore=0.013219
[LightGBM] [

In [40]:
y_pred = best_model.predict(X_val)
# 验证集的真实标签
y_true = y_val

In [41]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
print(confusion_matrix(y_true, y_pred))

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = best_model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.91      0.95      0.93      1457
           1       0.95      0.90      0.92      1381

    accuracy                           0.92      2838
   macro avg       0.93      0.92      0.92      2838
weighted avg       0.93      0.92      0.92      2838

[[1387   70]
 [ 144 1237]]
Accuracy: 0.9245947850599013
Precision: 0.926193030843816
Recall: 0.9238419038256722
F1 Score: 0.9243835460891183
ROC AUC: 0.9782279062301049
Specificity: 0.8490048043925875


In [42]:
# 获取特征重要性
feature_importances = best_model.feature_importances_

# 排序特征
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 打印特征重要性排序
print(importance_df)
print(importance_df.head(10))

           Feature  Importance
16            ZBMI         942
72       EQ5D5L_VS         762
93         Stress2         446
4           Income         386
94         Stress3         385
..             ...         ...
58  MedCompliance2          22
59  MedCompliance3          21
69       EQ_5D_5L3          19
63  MedCompliance7          18
60  MedCompliance4          12

[111 rows x 2 columns]
          Feature  Importance
16           ZBMI         942
72      EQ5D5L_VS         762
93        Stress2         446
4          Income         386
94        Stress3         385
92        Stress1         366
107      Chronic3         346
71      EQ_5D_5L5         325
26   Neuroticism2         315
109      Chronic5         313


In [43]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [44]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [45]:
# 使用训练好的模型对验证集进行预测
y_pred = model.predict(X_val)

# 验证集的真实标签
y_true = y_val

In [46]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
print(confusion_matrix(y_true, y_pred))

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = model.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.91      0.96      0.93      1457
           1       0.95      0.90      0.92      1381

    accuracy                           0.93      2838
   macro avg       0.93      0.93      0.93      2838
weighted avg       0.93      0.93      0.93      2838

[[1393   64]
 [ 140 1241]]
Accuracy: 0.9281183932346723
Precision: 0.9298168267464442
Recall: 0.9273491551435628
F1 Score: 0.9279116040811143
ROC AUC: 0.9796704167799387
Specificity: 0.8490048043925875


In [47]:
# 获取特征重要性
feature_importances = model.feature_importances_

# 排序特征
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 打印特征重要性排序
print(importance_df)
print(importance_df.head(10))

            Feature  Importance
107        Chronic3    0.063626
93          Stress2    0.050297
91          EBS_SF7    0.044737
109        Chronic5    0.043159
92          Stress1    0.041463
..              ...         ...
62   MedCompliance6    0.001541
69        EQ_5D_5L3    0.001539
64   MedCompliance8    0.001495
61   MedCompliance5    0.001430
57   MedCompliance1    0.001111

[111 rows x 2 columns]
       Feature  Importance
107   Chronic3    0.063626
93     Stress2    0.050297
91     EBS_SF7    0.044737
109   Chronic5    0.043159
92     Stress1    0.041463
94     Stress3    0.040757
108   Chronic4    0.032108
71   EQ_5D_5L5    0.029665
85     EBS_SF1    0.026491
90     EBS_SF6    0.023884


In [48]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 

In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
model = RandomForestClassifier()
model.fit(X_train, y_train)
# 定义参数网格
param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 设置网格搜索
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

# 执行网格搜索
grid_search.fit(X_train, y_train)

# 输出最佳参数
print("Best parameters:", grid_search.best_params_)

# 获取最佳模型
best_rf = grid_search.best_estimator_

Fitting 5 folds for each of 144 candidates, totalling 720 fits


D:\anaconda\envs\MentalAssessment\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


In [50]:
y_pred = best_rf.predict(X_val)
# 验证集的真实标签
y_true = y_val

In [51]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 打印分类报告
print(classification_report(y_true, y_pred))

# 打印混淆矩阵
conf_mat = confusion_matrix(y_true, y_pred)
print(conf_mat)

# 计算并打印准确率
print("Accuracy:", accuracy_score(y_true, y_pred))

# 计算其他性能指标
print("Precision:", precision_score(y_true, y_pred, average='macro'))
print("Recall:", recall_score(y_true, y_pred, average='macro'))
print("F1 Score:", f1_score(y_true, y_pred, average='macro'))

# 如果模型是二分类模型，并且你已经有了模型预测的概率，可以计算ROC AUC
y_scores = best_rf.predict_proba(X_val)[:, 1]  # 获取正类的概率
print("ROC AUC:", roc_auc_score(y_true, y_scores))

# 计算Specificity
tn, fp, fn, tp = conf_mat.ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

              precision    recall  f1-score   support

           0       0.90      0.96      0.93      1457
           1       0.95      0.89      0.92      1381

    accuracy                           0.93      2838
   macro avg       0.93      0.92      0.93      2838
weighted avg       0.93      0.93      0.93      2838

[[1394   63]
 [ 148 1233]]
Accuracy: 0.9256518675123326
Precision: 0.9277048205793341
Recall: 0.9247958741961824
F1 Score: 0.9254118201273167
ROC AUC: 0.9796612224835832
Specificity: 0.9567604667124228


In [52]:
# 获取特征重要性
feature_importances = best_rf.feature_importances_

# 排序特征
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 打印特征重要性排序
print(importance_df)
print(importance_df.head(10))

            Feature  Importance
93          Stress2    0.058531
107        Chronic3    0.056521
92          Stress1    0.046252
91          EBS_SF7    0.043841
109        Chronic5    0.041947
..              ...         ...
60   MedCompliance4    0.001537
64   MedCompliance8    0.001510
57   MedCompliance1    0.001430
63   MedCompliance7    0.001409
61   MedCompliance5    0.001338

[111 rows x 2 columns]
       Feature  Importance
93     Stress2    0.058531
107   Chronic3    0.056521
92     Stress1    0.046252
91     EBS_SF7    0.043841
109   Chronic5    0.041947
94     Stress3    0.036768
108   Chronic4    0.029238
71   EQ_5D_5L5    0.027665
89     EBS_SF5    0.026132
86     EBS_SF2    0.024098
